In [ ]:
import pandas as pd
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, Concatenate, Dropout, BatchNormalization, Add, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Configuration
tf.random.set_seed(42)
np.random.seed(42)

# ==============================================================================
# 1. CHARGEMENT ET PRÉPARATION
# ==============================================================================
print("Chargement des données...")
data = pd.read_csv('data/processed_data.csv')
data['date'] = pd.to_datetime(data['date'])

# Log-transform cible (Critique)
data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Définition des colonnes
ALL_COLS = [c for c in data.columns if c not in ['id', 'sales', 'is_train', 'date']]
# On retire 'family' des features car on va boucler dessus !
CAT_COLS = ['store_nbr', 'city', 'state', 'type', 'cluster', 'month', 'dayofweek'] 
NUM_COLS = [c for c in ALL_COLS if c not in CAT_COLS and c != 'family']

print(f"Features numériques : {len(NUM_COLS)}")
print(f"Features catégorielles : {len(CAT_COLS)}")

# ==============================================================================
# 2. PREPROCESSING GLOBAL
# ==============================================================================
print("Preprocessing...")

# Encodage des catégories
label_encoders = {}
for col in CAT_COLS:
    le = LabelEncoder()
    data[col] = data[col].astype(str)
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Scaling des numériques
scaler = StandardScaler()
data[NUM_COLS] = scaler.fit_transform(data[NUM_COLS])
data[NUM_COLS] = data[NUM_COLS].fillna(0)

# Liste des familles pour la boucle
FAMILIES = data['family'].unique()
print(f"Nombre de modèles à entraîner : {len(FAMILIES)}")

# ==============================================================================
# 3. DÉFINITION DU MODÈLE RESNET (Fonction)
# ==============================================================================
def build_resnet_model(num_features, cat_cols_info, label_encoders):
    inputs = []
    embeddings = []
    
    # Embeddings
    for col in cat_cols_info:
        vocab_size = len(label_encoders[col].classes_) + 1
        embed_dim = min(50, (vocab_size + 1) // 2)
        inp = Input(shape=(1,), name=f'input_{col}')
        inputs.append(inp)
        emb = Embedding(vocab_size, embed_dim)(inp)
        emb = Flatten()(emb)
        embeddings.append(emb)
    
    # Numériques
    input_num = Input(shape=(num_features,), name='input_numeric')
    inputs.append(input_num)
    
    # Fusion
    x = Concatenate()(embeddings + [input_num])
    
    # Projection
    x = Dense(128)(x) # Un peu plus petit car moins de données par famille
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    
    # Bloc Résiduel 1
    shortcut = x
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Add()([x, shortcut]) # Skip connection
    
    # Bloc Résiduel 2
    shortcut = x
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(64)(x) # Projection shortcut si dim change
    shortcut = Dense(64)(shortcut)
    x = Add()([x, shortcut])
    
    output = Dense(1, activation='linear', name='output')(x)
    
    model = Model(inputs=inputs, outputs=output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

# ==============================================================================
# 4. ENTRAÎNEMENT PAR FAMILLE (STRATÉGIE GAGNANTE)
# ==============================================================================
all_preds = []
test_ids_full = []

# On prépare le dataset de test global pour récupérer les IDs à la fin
test_global = data[data['is_train'] == 0].copy()

print("\nDébut de l'entraînement par famille...")

for fam in FAMILIES:
    print(f"\n--- Traitement de la famille : {fam} ---")
    
    # Filtrer les données pour cette famille
    df_fam = data[data['family'] == fam].copy()
    
    # Séparation Train/Val/Test pour cette famille
    train_df = df_fam[df_fam['is_train'] == 1]
    test_df = df_fam[df_fam['is_train'] == 0]
    
    # Validation temporelle (15 derniers jours)
    last_date = train_df['date'].max()
    val_start = last_date - pd.DateOffset(days=15)
    
    mask_train = train_df['date'] < val_start
    mask_val = train_df['date'] >= val_start
    
    # Préparation inputs Keras
    def get_inputs(df):
        X_num = df[NUM_COLS].values.astype('float32')
        X_cat = [df[c].values.astype('int32') for c in CAT_COLS]
        return X_cat + [X_num] # Liste [cat1, cat2..., num]
    
    X_train = get_inputs(train_df[mask_train])
    y_train = train_df.loc[mask_train, 'sales'].values.astype('float32')
    
    X_val = get_inputs(train_df[mask_val])
    y_val = train_df.loc[mask_val, 'sales'].values.astype('float32')
    
    X_test = get_inputs(test_df)
    ids_test = test_df['id'].values
    
    # Construction du modèle
    model = build_resnet_model(len(NUM_COLS), CAT_COLS, label_encoders)
    
    # Callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0)
    
    # Entraînement
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20, # 20 epochs suffisent souvent par famille
        batch_size=512, # Batch plus petit car moins de données
        callbacks=[es, rlr],
        verbose=0 # On réduit le bruit, affiche juste le résultat final
    )
    
    val_loss = min(history.history['val_loss'])
    print(f"Famille {fam} -> Meilleure Val MSE: {val_loss:.4f} (RMSE: {np.sqrt(val_loss):.4f})")
    
    # Prédiction
    preds_log = model.predict(X_test, batch_size=512, verbose=0).flatten()
    preds = np.expm1(preds_log)
    preds[preds < 0] = 0
    
    # Stockage
    df_res = pd.DataFrame({'id': ids_test, 'sales': preds})
    all_preds.append(df_res)
    
    # Nettoyage mémoire
    del model, X_train, X_val, X_test
    tf.keras.backend.clear_session()
    gc.collect()

# ==============================================================================
# 5. ASSEMBLAGE ET SOUMISSION
# ==============================================================================
print("\nAssemblage des prédictions...")
submission = pd.concat(all_preds).sort_values('id')
submission.to_csv('submission_resnet_per_family.csv', index=False)
print("Fichier 'submission_resnet_per_family.csv' généré avec succès !")
print(submission.head())